# **AutoJudge: Predicting Programming Problem Difficulty**

Online coding platforms such as **Codeforces**, **CodeChef**, and **Kattis** classify programming problems into difficulty levels and assign numerical difficulty scores. These labels are usually based on human judgment and user feedback.

---

## Objective

The goal of this project is to build an automated system that predicts:

- **Problem Class**: Easy / Medium / Hard *(Classification)*
- **Problem Score**: Numerical difficulty value *(Regression)*

The prediction is based **only on textual information**, including:

- Problem description  
- Input description  
- Output description  

---

## Dataset

Each data sample contains:

- **Title**
- **Description**
- **Input description**
- **Output description**
- **Problem class** (Easy / Medium / Hard)
- **Problem score** (0-10)

The dataset is provided and does **not require manual labeling**.

---

## Approach

- Clean and combine all text fields into a single input  
- Extract features using **TF-IDF** and handcrafted numerical features  
- Train a **classification model** to predict difficulty class  
- Train a **regression model** to predict difficulty score  

---

## Models Used

- **Classification**: Logistic Regression (Tried  Logistic Regression, Random Forest, SVM)  
- **Regression**: Gradient Boosting Regressor (Tried Linear Regression, Gradient Boosting, Random Forest also but it was very slow so not used it)  

---

## Evaluation

- **Classification**:
  - Accuracy
  - Confusion Matrix

- **Regression**:
  - Mean Absolute Error (MAE)
  - Root Mean Squared Error (RMSE)

---

## Web Interface

A simple **Streamlit-based web UI** allows users to:

- Paste problem description, input, and output  
- Click a **Predict** button  
- View predicted difficulty class and score  

No authentication or database is required.

## **Data Loading and Importing required Libraries**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
BASE_PATH = "/content/drive/MyDrive/ACM_Project"

data = f"{BASE_PATH}/data.jsonl"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
import joblib

In [ ]:
df = pd.read_json(data, lines=True)

## **Exploring Data**

In [ ]:
df.head()

,title,description,input_description,output_description,sample_io,problem_class,problem_score,url
0,Uuu,Unununium (Uuu) was the name of the chemical\n...,The input consists of one line with two intege...,The output consists of $M$ lines where the $i$...,"[{'input': '7 10', 'output': '1 2 2 3 1 3 3 4 ...",hard,9.7,https://open.kattis.com/problems/uuu
1,House Building,A number of eccentrics from central New York h...,"The input consists of $10$ test cases, which a...",Print $K$ lines with\n the positions of the...,"[{'input': '0 2 3 2 50 60 50 30 50 40', 'outpu...",hard,9.7,https://open.kattis.com/problems/husbygge
2,Mario or Luigi,Mario and Luigi are playing a game where they ...,,,"[{'input': '', 'output': ''}]",hard,9.6,https://open.kattis.com/problems/marioorluigi
3,The Wire Ghost,Žofka is bending a copper wire. She starts wit...,The first line contains two integers $L$ and $...,The output consists of a single line consistin...,"[{'input': '4 3 3 C 2 C 1 C', 'output': 'GHOST...",hard,9.6,https://open.kattis.com/problems/thewireghost
4,Barking Up The Wrong Tree,"Your dog Spot is let loose in the park. Well, ...",The first line of input consists of two intege...,Write a single line containing the length need...,"[{'input': '2 0 10 0 10 10', 'output': '14.14'...",hard,9.6,https://open.kattis.com/problems/barktree


In [ ]:
df['problem_class'].value_counts()

,count
problem_class,
hard,1941
medium,1405
easy,766


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4112 entries, 0 to 4111
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               4112 non-null   object 
 1   description         4112 non-null   object 
 2   input_description   4112 non-null   object 
 3   output_description  4112 non-null   object 
 4   sample_io           4112 non-null   object 
 5   problem_class       4112 non-null   object 
 6   problem_score       4112 non-null   float64
 7   url                 4112 non-null   object 
dtypes: float64(1), object(7)
memory usage: 257.1+ KB


In [ ]:
df.isnull().sum()

,0
title,0
description,0
input_description,0
output_description,0
sample_io,0
problem_class,0
problem_score,0
url,0


No missing values

In [ ]:
# dropped irrelevant columns
df = df.drop(columns=["title", "sample_io", "url"])
df.head()

,description,input_description,output_description,problem_class,problem_score
0,Unununium (Uuu) was the name of the chemical\n...,The input consists of one line with two intege...,The output consists of $M$ lines where the $i$...,hard,9.7
1,A number of eccentrics from central New York h...,"The input consists of $10$ test cases, which a...",Print $K$ lines with\n the positions of the...,hard,9.7
2,Mario and Luigi are playing a game where they ...,,,hard,9.6
3,Žofka is bending a copper wire. She starts wit...,The first line contains two integers $L$ and $...,The output consists of a single line consistin...,hard,9.6
4,"Your dog Spot is let loose in the park. Well, ...",The first line of input consists of two intege...,Write a single line containing the length need...,hard,9.6


In [ ]:
df['problem_score'].describe()

,problem_score
count,4112.000000
mean,5.114689
std,2.177770
min,1.100000
25%,3.300000
50%,5.200000
75%,6.900000
max,9.700000


Problem Score is in range of 1-10

## **Splitting Data into Train and Test**

In [ ]:
# Features
X = df[["description", "input_description", "output_description"]]

y_class = df["problem_class"]      # classification target
y_score = df["problem_score"]      # regression target

In [ ]:
# train test split
X_train, X_test, y_class_train, y_class_test, y_score_train, y_score_test = train_test_split(
    X,
    y_class,
    y_score,
    test_size=0.2,
    random_state=42,
    stratify=y_class
)

### Encoding Classes into numeric labels

In [ ]:
le = LabelEncoder()
y_class_train_encoded = le.fit_transform(y_class_train)
y_class_test_encoded = le.transform(y_class_test)

In [ ]:
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(label_mapping)

{'easy': np.int64(0), 'hard': np.int64(1), 'medium': np.int64(2)}


In [ ]:
print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

print("\nClass distribution in full data:")
print(y_class.value_counts(normalize=True))

print("\nClass distribution in train:")
print(y_class_train.value_counts(normalize=True))

print("\nClass distribution in test:")
print(y_class_test.value_counts(normalize=True))

Train size: (3289, 3)
Test size: (823, 3)

Class distribution in full data:
problem_class
hard      0.472033
medium    0.341683
easy      0.186284
Name: proportion, dtype: float64

Class distribution in train:
problem_class
hard      0.471876
medium    0.341745
easy      0.186379
Name: proportion, dtype: float64

Class distribution in test:
problem_class
hard      0.472661
medium    0.341434
easy      0.185905
Name: proportion, dtype: float64


## **Data Cleaning and Preprocessing**

### **Text Combination**
For each problem, the following fields are combined into a single text:
- Problem description
- Input description
- Output description

This combined text represents the full problem statement used for modeling.

In [ ]:
# combining description, input_description and output_description
def combine_text_columns(df):
    return df["description"] + " " + df["input_description"] + " " + df["output_description"]

In [ ]:
X_train["full_text"] = combine_text_columns(X_train)
X_test["full_text"] = combine_text_columns(X_test)

In [ ]:
X_train['full_text'][1855]

'In the 1984 Ghostbusters™ movie, the protagonists use proton\n    pack weapons that fire laser streams. This leads to the\n    following memorable dialog between scientists Peter Venkman and\n    Egon Spengler:\nSpengler: There’s something very important\n    I forgot to tell you.\nVenkman: What?\nSpengler: Don’t cross the streams.\nVenkman: Why?\nSpengler: It would be bad.\nVenkman: I’m fuzzy on the whole good/bad\n    thing. What do you mean, "bad"?\nSpengler: Try to imagine all life as you know\n    it stopping instantaneously and every molecule in your body\n    exploding at the speed of light.\nVenkman: Right. That’s bad. Okay. All right.\n    Important safety tip.\nIn the 30+ years since that time, there have been several\n    technical advances in their weapons systems:\nThe laser streams have been polarized, firing either\n        horizontally or vertically. There is no longer any danger\n        if streams having opposite polarity cross each other.\n        However, there wil

## **Feature Engineering & Text Representation**

### **1. Text Cleaning**
All textual fields are first normalized using a custom preprocessing function:
- Convert text to lowercase
- Remove HTML tags
- Remove URLs
- Normalize whitespace

This ensures consistency and reduces noise before feature extraction.

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)   # remove html
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # remove urls
    text = re.sub(r'\s+', ' ', text)  # replace multiple spaces/newlines/tabs with single space
    return text.strip()

### **2. Numeric Feature Extraction**
In addition to text embeddings, handcrafted numeric features are extracted to capture structural complexity:

**a. Text-based statistics**
- Log-transformed text length
- Log-transformed count of mathematical symbols

**b. Constraint-awareness features**
- Presence of constraints (e.g., ≤, ≤, constraints)
- Presence of large input sizes (e.g., 10⁵, 10⁶)
- Presence of time limit references

**c. Algorithm keyword frequencies**
Keyword counts (log-transformed) for major algorithmic categories:
- Dynamic Programming
- Graph Algorithms
- Data Structures
- Mathematics
- Geometry
- String Algorithms
- Greedy Techniques

These features help capture problem-solving difficulty patterns.

In [ ]:
def extract_numeric_features(text):
    text_lower = text.lower()

    # Algorithm groups
    algo_groups = {
        "dp": [
            "dp", "dynamic programming", "knapsack", "bitmask dp",
            "state", "transition", "memoization", "tabulation"
        ],
        "graph": [
            "graph", "bfs", "dfs", "dijkstra", "bellman ford",
            "floyd warshall", "topological", "shortest path",
            "flow", "max flow", "min cut", "matching",
            "strongly connected", "bridges", "articulation points",
            "tree", "lca"
        ],
        "ds": [
            "segment tree", "fenwick", "binary indexed tree",
            "heap", "priority queue", "stack", "queue",
            "deque", "union find", "disjoint set", "sparse table"
        ],
        "math": [
            "modulo", "prime", "gcd", "lcm", "combinatorics",
            "permutations", "probability", "matrix exponentiation",
            "fft", "fast fourier transform", "number theory"
        ],
        "geometry": [
            "geometry", "convex hull", "sweep line",
            "cross product", "dot product", "orientation"
        ],
        "string": [
            "string", "substring", "palindrome",
            "kmp", "z algorithm", "suffix array",
            "trie", "rolling hash"
        ],
        "greedy": [
            "greedy", "two pointers", "sliding window",
            "interval", "activity selection"
        ]
    }

    # Count per algorithm group
    group_counts = {}
    for group, keywords in algo_groups.items():
        count = sum(text_lower.count(k) for k in keywords)
        group_counts[f"{group}_count"] = np.log1p(count)

    # Math symbols
    math_symbols = "+-*/^=<>(){}[]|&!%"
    math_symbol_count = sum(text.count(sym) for sym in math_symbols)

    # Text length
    text_len = len(text)

    # Constraint-awareness features

    has_constraints = int("≤" in text or "<=" in text_lower or "constraints" in text_lower)

    has_big_n = int("10^5" in text or "10^6" in text or "10^7" in text or "10^" in text)

    has_time_limit = int("time limit" in text_lower or "seconds" in text_lower)

    return {
        "text_length": np.log1p(text_len),
        "math_symbol_count": np.log1p(math_symbol_count),
        "has_constraints": has_constraints,
        "has_big_n": has_big_n,
        "has_time_limit": has_time_limit,
        **group_counts
    }

In [ ]:
# Clean full_text column for TF-IDF and numeric features
X_train["full_text_clean"] = X_train["full_text"].apply(clean_text)
X_test["full_text_clean"] = X_test["full_text"].apply(clean_text)

In [ ]:
# Train numeric features
numeric_features_train = X_train["full_text_clean"].apply(extract_numeric_features)
numeric_df_train = pd.DataFrame(list(numeric_features_train))

# Test numeric features
numeric_features_test = X_test["full_text_clean"].apply(extract_numeric_features)
numeric_df_test = pd.DataFrame(list(numeric_features_test))

### **3. TF-IDF Feature Extraction**
The cleaned combined text is converted into numerical form using TF-IDF:
- Unigrams and bigrams
- Maximum 30,000 features
- Minimum document frequency = 5
- Sublinear term frequency scaling

This captures semantic and contextual information from the problem text.

In [ ]:
# creating tf-idf for data
tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features=30000, min_df=5, sublinear_tf=True, stop_words=None)
X_train_tfidf = tfidf.fit_transform(X_train["full_text_clean"])
X_test_tfidf = tfidf.transform(X_test["full_text_clean"])

### **4. Final Feature Matrix**
The final input features are created by horizontally stacking:
- TF-IDF vectors
- Extracted numeric features

This results in a unified representation combining semantic and structural signals.

**Final Shapes:**
- Training data: `(3289, 22991)`
- Test data: `(823, 22991)`

These features are used for both:
- Classification (Easy / Medium / Hard)
- Regression (numerical difficulty score)

In [ ]:
# making final data by combining tf-idf and numeric features
X_train_final = hstack([X_train_tfidf, numeric_df_train.values])
X_test_final = hstack([X_test_tfidf, numeric_df_test.values])

In [ ]:
print("X_train_final shape: ", X_train_final.shape)
print("X_test_final shape: ", X_test_final.shape)

X_train_final shape:  (3289, 22991)
X_test_final shape:  (823, 22991)


## **Testing some classification models**

Stratified K-Fold (5 splits) is used to maintain class balance across all cross-validation folds.

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### **Logistic Regression**

In [ ]:
lr_clf = LogisticRegression(max_iter=1000, random_state=42)

lr_cv_scores = cross_val_score(lr_clf, X_train_final, y_class_train_encoded, cv=cv, scoring='accuracy')

print("Logistic Regression CV Accuracy:", lr_cv_scores)
print("Mean CV Accuracy:", lr_cv_scores.mean())
print("Std Dev:", lr_cv_scores.std())

Logistic Regression CV Accuracy: [0.54255319 0.51519757 0.51215805 0.5106383  0.52663623]
Mean CV Accuracy: 0.5214366675456736
Std Dev: 0.011954637476514056


### **Random Forest Classifier**

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

rf_cv_scores = cross_val_score(rf_clf, X_train_final, y_class_train_encoded, cv=cv, scoring='accuracy')

print("\nRandom Forest CV Accuracy:", rf_cv_scores)
print("Mean CV Accuracy:", rf_cv_scores.mean())
print("Std Dev:", rf_cv_scores.std())


Random Forest CV Accuracy: [0.48480243 0.49544073 0.48024316 0.5106383  0.48554033]
Mean CV Accuracy: 0.49133299098323874
Std Dev: 0.010851320540513406


### **SVM**

In [ ]:
svm_clf = LinearSVC(random_state=42, max_iter=3000)

svm_cv_scores = cross_val_score(svm_clf, X_train_final, y_class_train_encoded, cv=5, scoring='accuracy', n_jobs=-1)

print("LinearSVC CV Accuracy:", svm_cv_scores)
print("Mean CV Accuracy:", svm_cv_scores.mean())
print("Std Dev:", svm_cv_scores.std())

LinearSVC CV Accuracy: [0.49392097 0.4893617  0.49696049 0.49392097 0.47488584]
Mean CV Accuracy: 0.48980999569749206
Std Dev: 0.007846795877960175


### **Selecting Logistic regression as final model and fine tuning it using GridSearchCV**

In [ ]:
param_grid = {
    "C": [0.01, 0.1, 1, 3, 10],
    "penalty": ["l2"],
    "class_weight": [None, "balanced"]
}

lr = LogisticRegression(max_iter=3000, solver="lbfgs", n_jobs=-1, random_state=42)

grid = GridSearchCV(lr, param_grid, cv=cv, scoring="accuracy", verbose=1, n_jobs=-1)

grid.fit(X_train_final, y_class_train_encoded)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=LogisticRegression(max_iter=3000, n_jobs=-1,
                                          random_state=42),
             n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 3, 10],
                         'class_weight': [None, 'balanced'],
                         'penalty': ['l2']},
             scoring='accuracy', verbose=1)

In [ ]:
print("Best CV Accuracy:", grid.best_score_)
print("Best Params:", grid.best_params_)

Best CV Accuracy: 0.520220862074549
Best Params: {'C': 1, 'class_weight': None, 'penalty': 'l2'}


In [ ]:
best_lr = grid.best_estimator_

In [ ]:
y_test_pred = best_lr.predict(X_test_final)

print("Test Accuracy:", accuracy_score(y_class_test_encoded, y_test_pred))
print("\nClassification Report:\n", classification_report(y_class_test_encoded, y_test_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_class_test_encoded, y_test_pred))

Test Accuracy: 0.5407047387606319

Classification Report:
               precision    recall  f1-score   support

           0       0.57      0.39      0.46       153
           1       0.57      0.82      0.67       389
           2       0.42      0.24      0.30       281

    accuracy                           0.54       823
   macro avg       0.52      0.48      0.48       823
weighted avg       0.52      0.54      0.51       823


Confusion Matrix:
 [[ 59  54  40]
 [ 16 319  54]
 [ 28 186  67]]


## **Classification Model Results**

The classification model was evaluated on the test set using accuracy, precision, recall, F1-score, and a confusion matrix.

**Overall Test Accuracy:** 54.07%

---

### Class-wise Performance

**Easy (0):**

- Precision: 0.57  
- Recall: 0.39  

The model struggles to correctly identify Easy problems, often confusing them with Medium or Hard.

**Hard (1):**

- Precision: 0.57  
- Recall: 0.82  

The model performs best on Hard problems, correctly identifying most of them.

**Medium (2):**

- Precision: 0.42  
- Recall: 0.24  

Medium problems are the hardest to classify, as they overlap with both Easy and Hard in textual complexity.

---

### Confusion Matrix Insights

- Many Medium problems are misclassified as Hard.  
- Easy problems are sometimes confused with Medium.  
- Hard problems are predicted most reliably.

---

### Conclusion

The results indicate that text-based difficulty prediction is challenging, especially for Medium-level problems. Despite moderate accuracy, the model successfully captures meaningful patterns and performs reasonably well without using deep learning, which aligns with the project guidelines.

## **Testing some regression models**

5-fold Stratified Cross-Validation is used to ensure that each fold preserves the original class distribution of Easy, Medium, and Hard problems.

In [ ]:
cv_reg = KFold(n_splits=5, shuffle=True, random_state=42)

### **Linear Regression**

In [ ]:
lr_reg = LinearRegression()

# RMSE
lr_rmse = -cross_val_score(lr_reg, X_train_final, y_score_train, cv=cv_reg, scoring="neg_root_mean_squared_error")

print("Linear Regression CV RMSE:", lr_rmse)
print("Mean RMSE:", lr_rmse.mean())
print("Std RMSE:", lr_rmse.std())

Linear Regression CV RMSE: [  2.48787881   2.74074997   2.19690558 156.39765489   2.15128787]
Mean RMSE: 33.194895424352055
Std RMSE: 61.60174791191362


### **Gradient Boosting**

In [ ]:
gb_reg = GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, max_depth=3, random_state=42)

gb_rmse = -cross_val_score(gb_reg, X_train_final, y_score_train, cv=cv_reg, scoring="neg_root_mean_squared_error")

print("Gradient Boosting CV RMSE:", gb_rmse)
print("Mean RMSE:", gb_rmse.mean())
print("Std RMSE:", gb_rmse.std())

Gradient Boosting CV RMSE: [2.01834257 2.02892498 2.0244672  1.98650478 1.98616861]
Mean RMSE: 2.0088816277191124
Std RMSE: 0.018712349243543255


## **Selecting Gradient Boosting as final model and fine tuning it using RandomizedSearchCV**

In [ ]:
cv_regg = KFold(n_splits=2, shuffle=True, random_state=42)

In [ ]:
param_dist = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.03, 0.05, 0.1],
    "max_depth": [2, 3, 4],
    "min_samples_leaf": [1, 5, 10]
}

gb = GradientBoostingRegressor(random_state=42)

rand_search = RandomizedSearchCV(gb_reg, param_distributions=param_dist, n_iter=5, cv=cv_regg, scoring="neg_root_mean_squared_error", n_jobs=-1, random_state=42, verbose=1)

rand_search.fit(X_train_final, y_score_train)

Fitting 2 folds for each of 5 candidates, totalling 10 fits


RandomizedSearchCV(cv=KFold(n_splits=2, random_state=42, shuffle=True),
                   estimator=GradientBoostingRegressor(learning_rate=0.05,
                                                       n_estimators=300,
                                                       random_state=42),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'learning_rate': [0.03, 0.05, 0.1],
                                        'max_depth': [2, 3, 4],
                                        'min_samples_leaf': [1, 5, 10],
                                        'n_estimators': [100, 200, 300]},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=1)

In [ ]:
print("Best RMSE:", -rand_search.best_score_)
print("Best Params:", rand_search.best_params_)

Best RMSE: 2.0360541156485006
Best Params: {'n_estimators': 200, 'min_samples_leaf': 5, 'max_depth': 4, 'learning_rate': 0.03}


## **Training Gradient Boosting on Full Train Data**

The model hyperparameters were selected based on cross-validation performance, as they provided good and stable results, without performing further randomized hyperparameter tuning.

In [ ]:
gb_final = GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, max_depth=3, random_state=42)

gb_final.fit(X_train_final, y_score_train)

GradientBoostingRegressor(learning_rate=0.05, n_estimators=300, random_state=42)

In [ ]:
y_pred = gb_final.predict(X_test_final)
rmse = np.sqrt(mean_squared_error(y_score_test, y_pred))
mae = mean_absolute_error(y_score_test, y_pred)
print("Test RMSE:", rmse)
print("Test MAE:", mae)

Test RMSE: 2.0092688066015807
Test MAE: 1.679500483848899


## **Regression Results (Gradient Boosting)**

The regression model achieved a Test RMSE of 2.01 and a Test MAE of 1.68, indicating a reasonable prediction accuracy for problem difficulty scores.

Since Gradient Boosting is an ensemble model that averages multiple weak learners, the predicted difficulty score may be slightly smoothed compared to the actual score. Therefore, exact matching with the true difficulty value is not expected, and small deviations are normal in NLP-based regression tasks.

---

#### All trained models and preprocessing objects were saved using joblib for deployment in the Streamlit web application.

In [ ]:
joblib.dump(tfidf, f"{BASE_PATH}/tfidf.pkl")
joblib.dump(best_lr, f"{BASE_PATH}/logreg_classifier.pkl")
joblib.dump(gb_final, f"{BASE_PATH}/gb_regressor.pkl")
joblib.dump(le, f"{BASE_PATH}/label_encoder.pkl")

['/content/drive/MyDrive/ACM_Project/label_encoder.pkl']